# Agent tutorial

This tutorial demonstrates AdalFlow agent features on using various tools and dealing with streaming.

## Prerequisites
1. Install AdalFlow: `pip install adalflow`
2. Set OpenAI API key: `export OPENAI_API_KEY="your-api-key-here"`

**Note**: This notebook requires an OpenAI API key to function properly.

In [ ]:
from IPython.display import clear_output 


!pip install adalflow pydantic openai

# Set OpenAI API key (replace with your actual key)
import os
os.environ["OPENAI_API_KEY"] = "your api key"
# Optional: Set Anthropic API key for Anthropic examples
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key-here"
clear_output()

## Setup and Imports

In [25]:
#Import required libraries
import os
import sys
import asyncio
from adalflow.components.agent.agent import Agent
from adalflow.components.agent.runner import Runner
from adalflow.components.model_client import OpenAIClient
from adalflow.components.model_client.anthropic_client import AnthropicAPIClient
from adalflow.core.func_tool import FunctionTool
from adalflow.core.types import (
    ToolOutput, 
    RawResponsesStreamEvent, 
    RunItemStreamEvent,
    ToolCallRunItem,
    ToolOutputRunItem,
    FinalOutputItem,
    ToolCallActivityRunItem
)
from adalflow.apps.cli_permission_handler import CLIPermissionHandler, AutoApprovalHandler
from adalflow.tracing import (
    set_tracing_disabled, 
    enable_mlflow_local, 
    trace
)
import adalflow as adal
from pathlib import Path
import os

# ENV_PATH = Path(os.getcwd()).parent.parent / ".env"
# print(ENV_PATH)

# # Setup environment
# adal.setup_env(ENV_PATH)

In [2]:
adal.__version__

'1.1.1'

## Hello World Agent

Our agent supports sync, async, sync generator and async generator functions whether it uses tooloutput or not.

In [26]:
# Demonstration of all 4 function types supported by AdalFlow agents

def calculator(expression: str) -> str:
    """A synchronous calculator that evaluates mathematical expressions."""
    try:
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error evaluating expression: {e}"

# 2. Async Function
async def web_search(query: str="what is the weather in SF today?") -> str:
    """web search on query."""
    # Simulate async operation
    await asyncio.sleep(0.5)
    return "San Francisco will be mostly cloudy today with some afternoon sun, reaching about 67 °F (20 °C) and dipping to 57 °F (14 °C) tonight."
    
# streaming tool can include toocallactivity run item and the last one as the final output observable to the llm agent
# 3. Sync Generator Function
def counter(limit: int):
    """A counter that counts up to a limit."""
    final_output = []
    for i in range(1, limit + 1):
        stream_item =  f"Count: {i}/{limit}"
        final_output.append(stream_item)
        stream_event = ToolCallActivityRunItem(data=stream_item)
        yield stream_event

    yield final_output

# 4. Async Generator Function
async def data_streamer(data_type: str):
    """Streams different types of data.

    Args:
        data_types can be "numbers", "letters", "words"
    """
    import asyncio
    data_items = {
        "numbers": [1, 2, 3, 4, 5],
        "letters": ["a", "b", "c", "d", "e"],
        "words": ["hello", "world", "from", "async", "generator"]
    }
    
    items = data_items.get(data_type, ["unknown"])
    final_output = []
    for item in items:
        await asyncio.sleep(0.2)  # Simulate async data fetching
        stream_item =  f"Streaming {data_type}: {item}"
        final_output.append(stream_item)
        stream_event = ToolCallActivityRunItem(data=stream_item)
        yield stream_event

    yield final_output


In [27]:
# setup the agent 

agent = Agent(
        name="StreamingAgent",
        tools=[FunctionTool(calculator), web_search, counter, data_streamer],
        model_client=OpenAIClient(),
        model_kwargs={"model": "gpt-4o", "temperature": 0.3},
        max_steps=5
    )

runner = Runner(agent=agent)

In [4]:
query = "What is 15 * 7 + 23? What is the sf weather, count to 10, and stream different types of data and tell me what is streamed. Must call existing tools."
 # query = "what is the sf weahter"

Sync call.

You can check the stepout where each is of `StepOutput.` Each function comes with `id`, and each step comes with `observation`, even if its an error.


In [5]:
# run call 


# Note: it should work on both sync and async fun normally, but cant work in the notebook, so we test only sync function 


agent = Agent(
        name="StreamingAgent",
        tools=[FunctionTool(calculator), counter],
        model_client=OpenAIClient(),
        model_kwargs={"model": "gpt-4o", "temperature": 0.3},
        max_steps=5
    )

runner = Runner(agent=agent)

def test():
    runner_result = runner.call( prompt_kwargs={"input_str": query})
    print(runner_result)
    for step in runner_result.step_history:
        print(step)
    print("Final Answer:")
    print(runner_result.answer)

test()

RunnerResult(step_history=[StepOutput(step=0, planner_prompt=None, action=Function(id='e785bb05-a74a-4870-b3fd-14de2397ac31', thought='First, I will calculate the expression 15 * 7 + 23.', name='calculator', args=[], kwargs={'expression': '15 * 7 + 23'}, _is_answer_final=False, _answer=None), function=Function(id='e785bb05-a74a-4870-b3fd-14de2397ac31', thought='First, I will calculate the expression 15 * 7 + 23.', name='calculator', args=[], kwargs={'expression': '15 * 7 + 23'}, _is_answer_final=False, _answer=None), observation='The result of 15 * 7 + 23 is 128', ctx=None), StepOutput(step=1, planner_prompt=None, action=Function(id='23785514-afa2-4c19-820a-75db7c76e8f3', thought='Next, I will count to 10 using the counter tool.', name='counter', args=[], kwargs={'limit': 10}, _is_answer_final=False, _answer=None), function=Function(id='23785514-afa2-4c19-820a-75db7c76e8f3', thought='Next, I will count to 10 using the counter tool.', name='counter', args=[], kwargs={'limit': 10}, _is_a

Acall behaves similar to call on the output data structure.

In [6]:
# run acall

agent = Agent(
        name="StreamingAgent",
        tools=[FunctionTool(calculator), web_search, counter, data_streamer],
        model_client=OpenAIClient(),
        model_kwargs={"model": "gpt-4o", "temperature": 0.3},
        max_steps=5
    )

runner = Runner(agent=agent)

runner_result = await runner.acall( prompt_kwargs={"input_str": query})

print(runner_result)
for step in runner_result.step_history:
    print(step)

print("Final Answer:")
print(runner_result.answer)

RunnerResult(step_history=[StepOutput(step=0, planner_prompt=None, action=Function(id='61696c5a-29b2-4295-a8bf-a01a8d1fa9e1', thought='First, calculate the expression 15 * 7 + 23.', name='calculator', args=[], kwargs={'expression': '15 * 7 + 23'}, _is_answer_final=False, _answer=None), function=Function(id='61696c5a-29b2-4295-a8bf-a01a8d1fa9e1', thought='First, calculate the expression 15 * 7 + 23.', name='calculator', args=[], kwargs={'expression': '15 * 7 + 23'}, _is_answer_final=False, _answer=None), observation='The result of 15 * 7 + 23 is 128', ctx=None), StepOutput(step=1, planner_prompt=None, action=Function(id='7d35933c-3554-4e2b-a850-46df87fcaa7a', thought='Next, perform a web search to find the weather in San Francisco.', name='web_search', args=[], kwargs={'query': 'what is the weather in SF today?'}, _is_answer_final=False, _answer=None), function=Function(id='7d35933c-3554-4e2b-a850-46df87fcaa7a', thought='Next, perform a web search to find the weather in San Francisco.',

In [7]:
# run async streaming with streaming events

async def run_astrem():
    streaming_result = runner.astream(
            prompt_kwargs={"input_str": query},
            model_kwargs={"stream": True}
    )

    # Process streaming events
    async for event in streaming_result.stream_events():
        if isinstance(event, RunItemStreamEvent):
            if isinstance(event.item, ToolCallRunItem):
                print(f"\n🔧 Calling tool: {event.item.data.name}")
                
            elif isinstance(event.item, ToolOutputRunItem):
                print(f"✅ Tool output: {event.item.data.output}")
                
            elif isinstance(event.item, ToolCallActivityRunItem):
                # This captures generator yields
                print(f"📝 Activity: {event.item.data}")
                
            elif isinstance(event.item, FinalOutputItem):
                print(f"\n🎯 Final answer: {event.item.data.answer}")

    return streaming_result

asyncio.run(run_astrem())


Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


function name calculator

🔧 Calling tool: calculator
✅ Tool output: The result of 15 * 7 + 23 is 128


Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


function name web_search

🔧 Calling tool: web_search


Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


...

Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


function name counter

🔧 Calling tool: counter
📝 Activity: Count: 1/10
📝 Activity: Count: 2/10
📝 Activity: Count: 3/10
📝 Activity: Count: 4/10
📝 Activity: Count: 5/10
📝 Activity: Count: 6/10
📝 Activity: Count: 7/10
📝 Activity: Count: 8/10
📝 Activity: Count: 9/10
📝 Activity: Count: 10/10
✅ Tool output: ['Count: 1/10', 'Count: 2/10', 'Count: 3/10', 'Count: 4/10', 'Count: 5/10', 'Count: 6/10', 'Count: 7/10', 'Count: 8/10', 'Count: 9/10', 'Count: 10/10']
function name data_streamer

🔧 Calling tool: data_streamer
📝 Activity: Streaming numbers: 1
📝 Activity: Streaming numbers: 2
📝 Activity: Streaming numbers: 3
📝 Activity: Streaming numbers: 4


Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


📝 Activity: Streaming numbers: 5
✅ Tool output: ['Streaming numbers: 1', 'Streaming numbers: 2', 'Streaming numbers: 3', 'Streaming numbers: 4', 'Streaming numbers: 5']

🎯 Final answer: The result of 15 * 7 + 23 is 128. San Francisco will be mostly cloudy today with some afternoon sun, reaching about 67 °F (20 °C) and dipping to 57 °F (14 °C) tonight. Counted to 10 successfully. Streamed numbers: 1, 2, 3, 4, 5.


RunnerStreamingResult(_event_queue=<Queue at 0x30fb4bfd0 maxsize=0 _queue=[RunItemStreamEvent(name='agent.execution_complete', item=FinalOutputItem(id='8c528fda-8f32-4c1c-b086-4562aad4866c', type='final_output', data=RunnerResult(step_history=[StepOutput(step=0, planner_prompt=None, action=Function(id='f5f7afb6-ae22-4fa6-9a1f-0facac1e389f', thought='First, I will calculate the expression 15 * 7 + 23.', name='calculator', args=[], kwargs={'expression': '15 * 7 + 23'}, _is_answer_final=False, _answer=None), function=Function(id='f5f7afb6-ae22-4fa6-9a1f-0facac1e389f', thought='First, I will calculate the expression 15 * 7 + 23.', name='calculator', args=[], kwargs={'expression': '15 * 7 + 23'}, _is_answer_final=False, _answer=None), observation='The result of 15 * 7 + 23 is 128', ctx=None), StepOutput(step=1, planner_prompt=None, action=Function(id='f5a711a1-fba7-4f13-80dd-a44e56f11253', thought='Next, I will search for the weather in San Francisco.', name='web_search', args=[], kwargs={'

### Streaming Raw Responses

This is the same with `astream` above with `delta` which streams the intermediate function call.

In [19]:
async def streaming_raw_responses_example():
    """Demonstrates handling raw response stream events."""
    print("\n=== Streaming Raw Responses Example ===")
    streaming_result = runner.astream(
        prompt_kwargs={"input_str": query},
        model_kwargs={"stream": True}
    )

    print("Raw streaming output:")
    last_event = None
    async for event in streaming_result.stream_events():
        if isinstance(event, RawResponsesStreamEvent):
            # print(f"\n🔄 Raw response event: {event.data}")
            # Process raw model output
            # check delta field 
            if hasattr(event.data, 'delta') and event.data.delta:
                delta = event.data.delta
                print(delta, end='', flush=True)
        last_event = event

    print("\n")  # Add newline after streaming

    # get the final runner result 
    print("\nFinal Result:")
    if hasattr(streaming_result, 'answer'):
        print(streaming_result.answer)
    else:
        print("No final answer available.")


    # step through the step history
    print("\nStep History:")
    for step in streaming_result.step_history:
        print(step)
    

    # get the final runner result
    print("\nFinal Runner Result:")
    print(last_event)
    return streaming_result

# Run the async example
await streaming_raw_responses_example()

Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.



=== Streaming Raw Responses Example ===
Raw streaming output:
{
    "thought": "First, I will calculate the expression 15 * 7 + 23.",
    "name": "calculator",
    "kwargs": {
        "expression": "15 * 7 + 23"
    },
    "_is_answer_final": false
}function name calculator


Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


```json
{
    "thought": "Next, I will check the weather in San Francisco.",
    "name": "web_search",
    "kwargs": {
        "query": "what is the weather in SF today?"
    },
    "_is_answer_final": false
}
```function name web_search


Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


...

Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


{
    "thought": "Finally, I will stream different types of data and report what is streamed.",
    "name": "data_streamer",
    "kwargs": {
        "data_type": "numbers"
    },
    "_is_answer_final": False
}function name data_streamer


Cannot track usage for generator/iterator. Usage tracking should be handled when consuming the stream.


{
    "thought": "I have completed all the tasks: calculated the expression, checked the weather, counted to 10, and streamed numbers.",
    "_is_answer_final": true,
    "_answer": "The result of 15 * 7 + 23 is 128. The weather in San Francisco is mostly cloudy with some afternoon sun, reaching about 67 °F (20 °C) and dipping to 57 °F (14 °C) tonight. Counted to 10 successfully. Streamed numbers: 1, 2, 3, 4, 5."
}


Final Result:
The result of 15 * 7 + 23 is 128. The weather in San Francisco is mostly cloudy with some afternoon sun, reaching about 67 °F (20 °C) and dipping to 57 °F (14 °C) tonight. Counted to 10 successfully. Streamed numbers: 1, 2, 3, 4, 5.

Step History:
StepOutput(step=0, planner_prompt=None, action=Function(id='34608cd1-724f-4b5b-a372-0f9f4868de3c', thought='First, I will calculate the expression 15 * 7 + 23.', name='calculator', args=[], kwargs={'expression': '15 * 7 + 23'}, _is_answer_final=False, _answer=None), function=Function(id='34608cd1-724f-4b5b-a372-0f9f

RunnerStreamingResult(_event_queue=<Queue at 0x31439add0 maxsize=0 _queue=[RunItemStreamEvent(name='agent.execution_complete', item=FinalOutputItem(id='c72e109d-851c-4c4c-a6fe-d8b1318013fc', type='final_output', data=RunnerResult(step_history=[StepOutput(step=0, planner_prompt=None, action=Function(id='34608cd1-724f-4b5b-a372-0f9f4868de3c', thought='First, I will calculate the expression 15 * 7 + 23.', name='calculator', args=[], kwargs={'expression': '15 * 7 + 23'}, _is_answer_final=False, _answer=None), function=Function(id='34608cd1-724f-4b5b-a372-0f9f4868de3c', thought='First, I will calculate the expression 15 * 7 + 23.', name='calculator', args=[], kwargs={'expression': '15 * 7 + 23'}, _is_answer_final=False, _answer=None), observation='The result of 15 * 7 + 23 is 128', ctx=None), StepOutput(step=1, planner_prompt=None, action=Function(id='3d37370f-5d55-4bab-8749-668c6d0794ad', thought='Next, I will check the weather in San Francisco.', name='web_search', args=[], kwargs={'query

### Streaming with Anthropic

The output works the same as OpenAI as we have standardized the output structure.

In [ ]:
async def streaming_anthropic_example():
    """Demonstrates streaming with Anthropic client."""
    print("\n=== Streaming Anthropic Example ===")
    
    # Check if Anthropic API key is available
    if not os.getenv("ANTHROPIC_API_KEY"):
        print("⚠️ ANTHROPIC_API_KEY not set, skipping Anthropic example")
        return None
    
    agent = Agent(
        name="AnthropicAgent",
        tools=[FunctionTool(calculator), web_search, counter, data_streamer],
        model_client=AnthropicAPIClient(),
        model_kwargs={"model": "claude-3-5-haiku-20241022", "stream": True, "temperature": 0.8},
        max_steps=5
    )

    runner = Runner(agent=agent)

    streaming_result = runner.astream(
        prompt_kwargs={"input_str": "Calculate 42 * 3 and explain why this might be significant"},
        model_kwargs={"stream": True}
    )

    last_event = None
    async for event in streaming_result.stream_events():
        if isinstance(event, RawResponsesStreamEvent):
            # print(f"\n🔄 Raw response event: {event.data}")
            # Process raw model output
            # check delta field 
            if hasattr(event.data, 'delta') and event.data.delta:
                delta = event.data.delta
                print(delta, end='', flush=True)
        last_event = event

    print("\n")  # Add newline after streaming

    # get the final runner result 
    print("\nFinal Result:")
    if hasattr(streaming_result, 'answer'):
        print(streaming_result.answer)
    else:
        print("No final answer available.")


    # step through the step history
    print("\nStep History:")
    for step in streaming_result.step_history:
        print(step)
    

    # get the final runner result
    print("\nFinal Runner Result:")
    print(last_event)

    return streaming_result

# Run the async example
await streaming_anthropic_example()


=== Streaming Anthropic Example ===
{
    "thought": "I'll first use the calculator to compute 42 * 3, and then I'll explore the potential significance of this number.",
    "name": "calculator",
    "kwargs": {
        "expression": "42 * 3"
    },
    "_is_answer_final": false
}function name calculator
{
    "thought": "I'll explore the significance of the number 126, drawing from cultural and mathematical references.",
    "name": "web_search",
    "kwargs": {"query": "Significance of the number 126 in culture, math, or science"},
    "_is_answer_final": false
}function name web_search
{
    "thought": "I noticed the previous web search didn't provide meaningful information about 126. I'll do a more targeted search about the number and its potential significance.",
    "name": "web_search",
    "kwargs": {"query": "Interesting mathematical or cultural significance of the number 126"},
    "_is_answer_final": false
}function name web_search
{
    "thought": "I'll first recall the ca

RunnerStreamingResult(_event_queue=<Queue at 0x3129826d0 maxsize=0 _queue=[RunItemStreamEvent(name='agent.execution_complete', item=FinalOutputItem(id='97a37a56-218d-4ccd-9048-20e88a6e1a10', type='final_output', data=RunnerResult(step_history=[StepOutput(step=0, planner_prompt=None, action=Function(id='b5a6fb52-e40a-4125-85d8-462c50bef08d', thought="I'll first use the calculator to compute 42 * 3, and then I'll explore the potential significance of this number.", name='calculator', args=[], kwargs={'expression': '42 * 3'}, _is_answer_final=False, _answer=None), function=Function(id='b5a6fb52-e40a-4125-85d8-462c50bef08d', thought="I'll first use the calculator to compute 42 * 3, and then I'll explore the potential significance of this number.", name='calculator', args=[], kwargs={'expression': '42 * 3'}, _is_answer_final=False, _answer=None), observation='The result of 42 * 3 is 126', ctx=None), StepOutput(step=1, planner_prompt=None, action=Function(id='467f1b7e-5eda-4990-b454-1d08130c